In [9]:
import requests
import pandas as pd
import time
import numpy as np

API_KEY = 'API_KEY'

# Step 1: Define grid over the Canton of Zurich
lat_range = np.arange(47.2, 47.71, 0.1)  # ~11 km spacing
lon_range = np.arange(8.4, 9.11, 0.1)
locations = [(round(lat, 4), round(lon, 4)) for lat in lat_range for lon in lon_range]

# Step 2: Define keywords (supermarket brands)
keywords = ["Spar"]
radius = 10000  # 10 km

# Step 3: Function to get places from Google Places API with pagination
def get_places(location, radius, keyword):
    url = (
        f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
        f'location={location}&radius={radius}&keyword={keyword}&type=supermarket&key={API_KEY}'
    )
    places = []
    while url:
        response = requests.get(url)
        data = response.json()
        places.extend(data.get('results', []))
        next_page_token = data.get('next_page_token')

        if next_page_token:
            time.sleep(2)  # wait for token to become valid
            url = (
                f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
                f'pagetoken={next_page_token}&key={API_KEY}'
            )
        else:
            url = None
    return places

# Step 4: Query all grid points and keywords
results = []
for lat, lon in locations:
    loc_str = f"{lat},{lon}"
    for keyword in keywords:
        print(f"🔍 Searching for {keyword} near {loc_str}...")
        try:
            places = get_places(loc_str, radius, keyword)
            for place in places:
                results.append({
                    'name': place.get('name'),
                    'address': place.get('vicinity'),
                    'lat': place['geometry']['location']['lat'],
                    'lng': place['geometry']['location']['lng'],
                    'type': keyword
                })
        except Exception as e:
            print(f"⚠️ Error at {loc_str} for {keyword}: {e}")
        time.sleep(1)  # avoid hitting rate limits

# Step 5: Save to CSV
df = pd.DataFrame(results).drop_duplicates()
df.to_csv('zurich_canton_spar.csv', index=False)


🔍 Searching for Spar near 47.2,8.4...
🔍 Searching for Spar near 47.2,8.5...
🔍 Searching for Spar near 47.2,8.6...
🔍 Searching for Spar near 47.2,8.7...
🔍 Searching for Spar near 47.2,8.8...
🔍 Searching for Spar near 47.2,8.9...
🔍 Searching for Spar near 47.2,9.0...
🔍 Searching for Spar near 47.2,9.1...
🔍 Searching for Spar near 47.3,8.4...
🔍 Searching for Spar near 47.3,8.5...
🔍 Searching for Spar near 47.3,8.6...
🔍 Searching for Spar near 47.3,8.7...
🔍 Searching for Spar near 47.3,8.8...
🔍 Searching for Spar near 47.3,8.9...
🔍 Searching for Spar near 47.3,9.0...
🔍 Searching for Spar near 47.3,9.1...
🔍 Searching for Spar near 47.4,8.4...
🔍 Searching for Spar near 47.4,8.5...
🔍 Searching for Spar near 47.4,8.6...
🔍 Searching for Spar near 47.4,8.7...
🔍 Searching for Spar near 47.4,8.8...
🔍 Searching for Spar near 47.4,8.9...
🔍 Searching for Spar near 47.4,9.0...
🔍 Searching for Spar near 47.4,9.1...
🔍 Searching for Spar near 47.5,8.4...
🔍 Searching for Spar near 47.5,8.5...
🔍 Searching 

In [7]:
import pandas as pd

# Load the cleaned competitor supermarkets dataset
df_competitors = pd.read_csv('zurich_canton_supermarkets_clean_split.csv')

# Load the cleaned SPAR dataset
df_spar = pd.read_csv('zurich_canton_spar_clean.csv')

# Add 'type' column to SPAR dataset if not already present
df_spar['type'] = 'Spar'

# Ensure consistent columns between both datasets
common_columns = ['name', 'address', 'lat', 'lng', 'street', 'city', 'type']

# Combine both DataFrames
df_combined = pd.concat([df_competitors[common_columns], df_spar[common_columns]], ignore_index=True)

# Save to a new CSV file
df_combined.to_csv('zurich_canton_all_supermarkets.csv', index=False)

print("✅ Combined dataset saved as 'zurich_canton_all_supermarkets.csv'")


✅ Combined dataset saved as 'zurich_canton_all_supermarkets.csv'
